In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [20]:
# -------------------------------
# ✅ STEP 0: Change these
# -------------------------------
DATASET_FOLDER = "/content/drive/MyDrive/TestDataSet"      # <-- CHANGE to your dataset folder name
LABELS_JSON    = "/content/drive/MyDrive/TestDataSet/labels_list.json"       # <-- CHANGE to your label JSON file

In [22]:
import os
import json
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

# -------------------------------
# STEP 1: Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------------------
# STEP 2: Load model
# -------------------------------
model = torchvision.models.resnet34(weights="IMAGENET1K_V1")
model.eval()
model.to(device)

# -------------------------------
# STEP 3: Load and normalize data
# -------------------------------
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms  = np.array([0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

dataset = torchvision.datasets.ImageFolder(root=DATASET_FOLDER, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# -------------------------------
# STEP 4: Parse labels JSON
# -------------------------------
with open(LABELS_JSON, "r") as f:
    raw = json.load(f)

# Build:
# - dataset label idx (0–99) → ImageNet idx (401–500)
# - ImageNet idx → class_name
# - ImageNet idx → new class ID (1–100)
imagenet_indices = []
name_map = {}
id_map = {}

for i, entry in enumerate(raw):
    idx_str, class_name = entry.split(":", 1)
    imagenet_id = int(idx_str.strip())         # 401 → 500
    imagenet_indices.append(imagenet_id)       # ordered list
    name_map[imagenet_id] = class_name.strip() # 401 → "accordion"
    id_map[imagenet_id] = i + 1                # 401 → 1, 402 → 2, ..., 500 → 100

# dataset label → ImageNet index (401–500)
label_map = {i: imagenet_indices[i] for i in range(len(imagenet_indices))}

# -------------------------------
# STEP 5: Evaluate accuracy
# -------------------------------
top1, top5, total = 0, 0, 0
printed = 0
print_limit = 5

with torch.no_grad():
    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = outputs.topk(5, dim=1).indices  # shape: [batch_size, 5]

        for i in range(images.size(0)):
            dataset_label = labels[i].item()             # 0–99
            true_imagenet = label_map[dataset_label]     # e.g. 401
            true_class_id = id_map[true_imagenet]        # e.g. 1–100

            top5_preds = preds[i].tolist()
            top5_class_ids = [id_map[p] for p in top5_preds if p in id_map]

            if len(top5_class_ids) == 0:
                continue  # skip if none of the top5 are in our 100-class subset

            if top5_class_ids[0] == true_class_id:
                top1 += 1
            if true_class_id in top5_class_ids:
                top5 += 1
            total += 1

            if printed < print_limit:
                print(f"Example {printed+1}: GT={true_class_id:3d} ({name_map[true_imagenet]}) | "
                      f"Pred={top5_class_ids[0]:3d} ({name_map.get(top5_preds[0], '[Unknown]')}) | "
                      f"Top5={top5_class_ids}")
                printed += 1

# -------------------------------
# STEP 6: Final results
# -------------------------------
print(f"\n✅ Evaluated {total} images")
print(f"Top-1 Accuracy: {top1 / total * 100:.2f}%")
print(f"Top-5 Accuracy: {top5 / total * 100:.2f}%")


Using device: cuda


 12%|█▎        | 2/16 [00:00<00:02,  5.46it/s]

Example 1: GT=  1 (accordion) | Pred=  1 (accordion) | Top5=[1]
Example 2: GT=  1 (accordion) | Pred=  1 (accordion) | Top5=[1]
Example 3: GT=  1 (accordion) | Pred=  1 ([Unknown]) | Top5=[1]
Example 4: GT=  1 (accordion) | Pred=  1 (accordion) | Top5=[1]
Example 5: GT=  2 (acoustic guitar) | Pred=  2 ([Unknown]) | Top5=[2, 20, 86]


100%|██████████| 16/16 [00:03<00:00,  4.90it/s]


✅ Evaluated 484 images
Top-1 Accuracy: 90.91%
Top-5 Accuracy: 97.31%
